# Donor Modeling Pipeline - Production Ready

## Overview

This notebook provides a complete, configurable modeling pipeline for donor behavior prediction:

- **Configuration-driven**: All settings in YAML files
- **Feature selection**: Per-cohort, per-label automatic selection
- **CV grid search**: Adaptive hyperparameter tuning based on label characteristics
- **Multiple models**: LogReg, LGBM, Ridge, Tweedie (for revenue)
- **Optional ensemble**: Weighted averaging with SHAP on best model
- **Threshold optimization**: Find optimal cutoffs for business objectives
- **Full explainability**: SHAP values for best model

## Quick Start

1. Edit **Section 0** (Runtime Configuration) with your label and cohorts
2. Run all cells
3. Results saved to `./models/<label>/<cohort>/best_model/`

---
# Section 0: Runtime Configuration
**Edit this cell for each modeling run**

In [1]:
# ============================================================================
# RUNTIME CONFIGURATION - Edit this cell to iterate quickly
# ============================================================================

# --- Quick Settings (change these frequently) ---
LABEL = 'label_will_give_during_giving_tuesday'

# Define ALL cohorts in cohorts.yaml, but only run these:
COHORTS_TO_RUN = ['all_donors']  # or None for all defined cohorts

# Which models to train (None = all enabled in models.yaml)
MODELS_TO_RUN = None  # or ['logreg', 'lgbm'] to specify

# Enable ensemble?
ENABLE_ENSEMBLE = False

# Manual feature exclusions (optional)
MANUAL_EXCLUDE_FEATURES = [
    'days_since_last_email_sent',
    'emails_clicked_3m',
    'emails_opened_3m',
    'emails_clicked_12m',
    'emails_opened_12m',
    'email_click_rate_3m',
    'email_click_rate_12m',
    'emails_sent_3m',
    'emails_sent_12m',
]

# --- Data Paths ---
TRAIN_FEATURES_PATH = '/Users/matt.fritz/Desktop/train_features_all_last3yr_365lag.csv'
OOT_FEATURES_PATH = '/Users/matt.fritz/Desktop/oot_features_all_last3yr_365lag.csv'

# --- Config Directory ---
CONFIG_DIR = '/Users/matt.fritz/Desktop/modeling_config'

# --- Output Settings ---
OUTPUT_BASE_DIR = '/Users/matt.fritz/Desktop/modeling_results'
SAVE_CANDIDATE_MODELS = False  # Only save best model by default
VERBOSE = True

print("✓ Runtime configuration set")
print(f"  Label: {LABEL}")
print(f"  Cohorts: {COHORTS_TO_RUN or 'ALL'}")
print(f"  Models: {MODELS_TO_RUN or 'ALL ENABLED'}")
print(f"  Ensemble: {ENABLE_ENSEMBLE}")

✓ Runtime configuration set
  Label: label_will_give_during_giving_tuesday
  Cohorts: ['all_donors']
  Models: ALL ENABLED
  Ensemble: False


---
# Section 1: Imports & Setup

In [2]:
# ============================================================================
# IMPORTS & SETUP
# ============================================================================

import os
import json
import pickle
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any

import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge, TweedieRegressor
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss, log_loss,
    mean_squared_error, mean_absolute_error, r2_score,
    roc_curve, precision_recall_curve, f1_score,
    precision_score, recall_score
)

import lightgbm as lgb
import shap

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
sns.set_style('whitegrid')

print("✓ Imports loaded")

✓ Imports loaded


---
# Section 2: Configuration Loading

In [3]:
# ============================================================================
# CONFIGURATION LOADING
# ============================================================================

def load_yaml_config(filepath: str) -> dict:
    """Load YAML configuration file."""
    with open(filepath, 'r') as f:
        return yaml.safe_load(f)

# Load all configuration files
config_files = {
    'models': f'{CONFIG_DIR}/models.yaml',
    'feature_selection': f'{CONFIG_DIR}/feature_selection.yaml',
    'evaluation': f'{CONFIG_DIR}/evaluation.yaml',
    'cohorts': f'{CONFIG_DIR}/cohorts.yaml'
}

configs = {}
for name, filepath in config_files.items():
    if not os.path.exists(filepath):
        print(f"⚠️  Config file not found: {filepath}")
        print(f"   Using default settings for {name}")
        configs[name] = {}
    else:
        configs[name] = load_yaml_config(filepath)
        print(f"✓ Loaded {name} config")

# Extract for convenience
MODEL_CONFIG = configs['models']
FEATURE_CONFIG = configs['feature_selection']
EVAL_CONFIG = configs['evaluation']
COHORT_CONFIG = configs['cohorts']

print("\n✓ All configurations loaded")

✓ Loaded models config
✓ Loaded feature_selection config
✓ Loaded evaluation config
✓ Loaded cohorts config

✓ All configurations loaded


---
# Section 3: Data Preparation Functions

In [4]:
# ============================================================================
# DATA PREPARATION FUNCTIONS
# ============================================================================

def load_data(train_path: str, oot_path: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load training and OOT feature datasets."""
    train = pd.read_csv(train_path)
    oot = pd.read_csv(oot_path)
    return train, oot


def apply_cohort_filter(df: pd.DataFrame, cohort_name: str, 
                       cohort_config: dict) -> pd.DataFrame:
    """
    Apply cohort filters from configuration.
    
    Cohort config format:
    {
        'description': '...',
        'filters': {
            'column_name': {
                'operator': '==', '>', '<', '>=', '<=', '!=', 'not_null', 'between'
                'value': <value> or [low, high] for between
            }
        }
    }
    """
    if cohort_name not in cohort_config['cohorts']:
        raise ValueError(f"Cohort '{cohort_name}' not found in config")
    
    cohort_def = cohort_config['cohorts'][cohort_name]
    filters = cohort_def.get('filters', {})
    
    mask = pd.Series(True, index=df.index)
    
    for col, spec in filters.items():
        if col not in df.columns:
            print(f"⚠️  Column '{col}' not found, skipping filter")
            continue
        
        col_data = df[col]
        operator = spec['operator']
        
        if operator == 'not_null':
            mask &= col_data.notnull()
        elif operator == 'is_null':
            mask &= col_data.isnull()
        elif operator == 'between':
            value = spec['value']
            mask &= (col_data >= value[0]) & (col_data <= value[1])
        elif operator == '==':
            mask &= col_data == spec['value']
        elif operator == '!=':
            mask &= col_data != spec['value']
        elif operator == '>':
            mask &= col_data > spec['value']
        elif operator == '>=':
            mask &= col_data >= spec['value']
        elif operator == '<':
            mask &= col_data < spec['value']
        elif operator == '<=':
            mask &= col_data <= spec['value']
    
    filtered = df[mask].copy()
    
    if VERBOSE:
        print(f"  Cohort '{cohort_name}': {len(filtered):,} donors "
              f"({len(filtered)/len(df)*100:.1f}%)")
    
    return filtered


def split_features_labels(df: pd.DataFrame, label: str, 
                         id_col: str = 'donor_id') -> Tuple[pd.DataFrame, pd.Series, pd.Series]:
    """
    Split dataframe into features, labels, and IDs.
    Automatically excludes:
    - donor_id
    - date columns (first_donation_date, last_donation_date)
    - ALL label_* columns (including the target label)
    """
    # Get donor IDs
    if id_col in df.columns:
        donor_ids = df[id_col].copy()
    else:
        donor_ids = pd.Series(range(len(df)), name='donor_id')
    
    # Get label FIRST (before we drop it)
    if label not in df.columns:
        raise ValueError(f"Label '{label}' not found in dataframe")
    y = df[label].copy()
    
    # Exclude columns
    exclude_cols = [id_col]
    
    # Exclude date columns
    date_cols = ['first_donation_date', 'last_donation_date']
    exclude_cols.extend([c for c in date_cols if c in df.columns])
    
    # Exclude ALL label_* columns (including the target label)
    # We already extracted y above, so we can safely drop all label columns
    label_cols = [c for c in df.columns if c.startswith('label_')]
    exclude_cols.extend(label_cols)  # ← THIS LINE WAS MISSING!

    # Manually exclude specific features
    try:
        from __main__ import MANUAL_EXCLUDE_FEATURES
        exclude_cols.extend([c for c in MANUAL_EXCLUDE_FEATURES if c in df.columns])
    except ImportError:
        pass  # if not defined, do nothing

    # Get features
    X = df.drop(columns=exclude_cols, errors='ignore')
    
    if VERBOSE:
        print(f"  Features: {X.shape[1]} columns")
        print(f"  Labels: {y.shape[0]} samples")
        print(f"  Excluded: {len(exclude_cols)} columns ({len(label_cols)} label columns)")
    
    return X, y, donor_ids


def detect_task_type(y: pd.Series) -> str:
    """Detect if task is classification or regression."""
    unique_vals = y.dropna().unique()
    
    # Binary classification: only 0 and 1
    if len(unique_vals) <= 2 and set(unique_vals).issubset({0, 1}):
        return 'classification'
    
    # Regression: continuous or many unique values
    if len(unique_vals) > 10:
        return 'regression'
    
    # If unclear, check dtype
    if pd.api.types.is_numeric_dtype(y) and not pd.api.types.is_integer_dtype(y):
        return 'regression'
    
    return 'classification'


print("✓ Data preparation functions defined")

✓ Data preparation functions defined


---
# Section 4: Feature Selection Functions

In [5]:
# ============================================================================
# FEATURE SELECTION FUNCTIONS
# ============================================================================

def remove_correlated_features(X: pd.DataFrame, threshold: float = 0.95) -> List[str]:
    """
    Remove highly correlated features.
    Returns list of features to keep.
    """
    # Only numeric features
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    
    if len(numeric_cols) == 0:
        return X.columns.tolist()
    
    # Calculate correlation matrix
    corr_matrix = X[numeric_cols].corr().abs()
    
    # Find pairs above threshold
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features to drop
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    features_to_keep = [c for c in X.columns if c not in to_drop]
    
    if VERBOSE and len(to_drop) > 0:
        print(f"  Removed {len(to_drop)} correlated features (>{threshold})")
    
    return features_to_keep


def select_features_tree_importance(X: pd.DataFrame, y: pd.Series,
                                   top_n: int = None,
                                   cumulative_threshold: float = 0.95,
                                   min_features: int = 20,
                                   max_features: int = 100) -> Tuple[List[str], pd.DataFrame]:
    """
    Select features using tree-based importance.
    
    Returns:
        features_to_keep: List of selected feature names
        importance_df: DataFrame with feature importance scores
    """
    # Quick LightGBM model for feature selection
    task_type = detect_task_type(y)
    
    if task_type == 'classification':
        model = lgb.LGBMClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            num_leaves=15,
            random_state=42,
            verbose=-1
        )
    else:
        model = lgb.LGBMRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            num_leaves=15,
            random_state=42,
            verbose=-1
        )
    
    # Handle categorical features
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    X_clean = X.copy()
    
    # Simple encoding for feature selection
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            X_clean[col] = pd.Categorical(X_clean[col]).codes
    
    # Fill NaNs
    X_clean = X_clean.fillna(-999)
    
    # Fit model
    model.fit(X_clean, y)
    
    # Get importance
    importance = model.feature_importances_
    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    # Determine features to keep
    if top_n is not None:
        features_to_keep = importance_df.head(top_n)['feature'].tolist()
    else:
        # Use cumulative importance
        importance_df['cum_importance'] = (
            importance_df['importance'].cumsum() / importance_df['importance'].sum()
        )
        
        # Get features explaining cumulative_threshold of importance
        n_features = (importance_df['cum_importance'] <= cumulative_threshold).sum() + 1
        
        # Apply min/max constraints
        n_features = max(min_features, min(n_features, max_features))
        
        features_to_keep = importance_df.head(n_features)['feature'].tolist()
    
    if VERBOSE:
        print(f"  Selected {len(features_to_keep)} features via tree importance")
        print(f"  Top 5 features: {', '.join(features_to_keep[:5])}")
    
    return features_to_keep, importance_df


def remove_low_variance_features(X: pd.DataFrame, threshold: float = 0.01) -> List[str]:
    """Remove features with variance below threshold."""
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    
    variances = X[numeric_cols].var()
    keep_cols = variances[variances >= threshold].index.tolist()
    
    # Keep all categorical columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    features_to_keep = keep_cols + categorical_cols
    
    removed = len(numeric_cols) - len(keep_cols)
    if VERBOSE and removed > 0:
        print(f"  Removed {removed} low-variance features (<{threshold})")
    
    return features_to_keep


def run_feature_selection(X: pd.DataFrame, y: pd.Series, 
                         config: dict) -> Tuple[pd.DataFrame, dict]:
    """
    Run complete feature selection pipeline.
    
    Returns:
        X_selected: DataFrame with selected features
        selection_info: Dict with selection metadata
    """
    if not config.get('enabled', True):
        return X, {'method': 'none', 'n_features': X.shape[1]}
    
    if VERBOSE:
        print(f"\nFeature Selection Pipeline:")
        print(f"  Starting features: {X.shape[1]}")
    
    selection_info = {
        'initial_features': X.shape[1],
        'stages': []
    }
    
    features_to_keep = X.columns.tolist()
    importance_df = None
    
    # Stage 1: Correlation filter
    if config.get('stages', {}).get('correlation_filter', {}).get('enabled', False):
        threshold = config['stages']['correlation_filter']['threshold']
        features_to_keep = remove_correlated_features(X[features_to_keep], threshold)
        selection_info['stages'].append({
            'name': 'correlation_filter',
            'n_features_kept': len(features_to_keep)
        })
    
    # Stage 2: Tree importance
    if config.get('stages', {}).get('tree_importance', {}).get('enabled', True):
        tree_config = config['stages']['tree_importance']
        auto_config = tree_config.get('auto_select', {})
        
        features_to_keep, importance_df = select_features_tree_importance(
            X[features_to_keep], y,
            top_n=tree_config.get('top_n_features'),
            cumulative_threshold=auto_config.get('threshold', 0.95),
            min_features=auto_config.get('min_features', 20),
            max_features=auto_config.get('max_features', 100)
        )
        selection_info['stages'].append({
            'name': 'tree_importance',
            'n_features_kept': len(features_to_keep)
        })
    
    # Stage 3: Variance filter
    if config.get('stages', {}).get('variance_filter', {}).get('enabled', True):
        threshold = config['stages']['variance_filter']['threshold']
        features_to_keep = remove_low_variance_features(X[features_to_keep], threshold)
        selection_info['stages'].append({
            'name': 'variance_filter',
            'n_features_kept': len(features_to_keep)
        })
    
    X_selected = X[features_to_keep].copy()
    selection_info['final_features'] = len(features_to_keep)
    selection_info['feature_names'] = features_to_keep
    selection_info['importance_scores'] = importance_df
    
    if VERBOSE:
        print(f"  Final features: {X_selected.shape[1]}")
    
    return X_selected, selection_info


print("✓ Feature selection functions defined")

✓ Feature selection functions defined


---
# Section 5: Model Building Functions

In [6]:
# ============================================================================
# MODEL BUILDING FUNCTIONS
# ============================================================================

def identify_feature_types(X: pd.DataFrame) -> Tuple[List[str], List[str]]:
    """Identify numeric and categorical features."""
    numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    return numeric_features, categorical_features


def create_preprocessor(numeric_features: List[str], 
                       categorical_features: List[str]) -> ColumnTransformer:
    """Create sklearn preprocessing pipeline."""
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop'
    )
    
    return preprocessor


def get_adaptive_param_grid(model_name: str, task_type: str, 
                           label_sparsity: float, n_samples: int,
                           config: dict) -> dict:
    """
    Get hyperparameter grid based on data characteristics.
    
    label_sparsity: Proportion of positive class (classification) or non-zero (regression)
    n_samples: Number of training samples
    """
    task_config = config.get(task_type, {})
    model_config = task_config.get(model_name, {})
    cv_config = model_config.get('cv_search', {})
    
    if not cv_config.get('enabled', False):
        return {}
    
    adaptive_rules = cv_config.get('adaptive_rules', {})
    
    # Determine which rule to use for LGBM
    if 'lgbm' in model_name:
        # Check small sample
        small_sample_threshold = adaptive_rules.get('small_sample', {}).get('threshold', 10000)
        
        if n_samples < small_sample_threshold:
            rule = adaptive_rules.get('small_sample', {})
            if VERBOSE:
                print(f"    Using 'small_sample' grid (n={n_samples:,})")
        
        # Classification: check sparse_label vs dense_label
        elif task_type == 'classification':
            sparse_threshold = adaptive_rules.get('sparse_label', {}).get('threshold', 0.05)
            dense_threshold = adaptive_rules.get('dense_label', {}).get('threshold', 0.05)
            
            if label_sparsity < sparse_threshold:
                rule = adaptive_rules.get('sparse_label', {})
                if VERBOSE:
                    print(f"    Using 'sparse_label' grid (prevalence={label_sparsity:.2%})")
            elif label_sparsity >= dense_threshold:
                rule = adaptive_rules.get('dense_label', {})
                if VERBOSE:
                    print(f"    Using 'dense_label' grid (prevalence={label_sparsity:.2%})")
            else:
                rule = adaptive_rules.get('default', {})
                if VERBOSE:
                    print(f"    Using 'default' grid")
        
        # Regression: check sparse_outcomes
        else:  # task_type == 'regression'
            sparse_threshold = adaptive_rules.get('sparse_outcomes', {}).get('threshold', 0.10)
            
            if label_sparsity < sparse_threshold:
                rule = adaptive_rules.get('sparse_outcomes', {})
                if VERBOSE:
                    print(f"    Using 'sparse_outcomes' grid (non-zero={label_sparsity:.2%})")
            else:
                rule = adaptive_rules.get('default', {})
                if VERBOSE:
                    print(f"    Using 'default' grid")
        
        param_grid = rule.get('param_grid', {})
    else:
        # Non-LGBM models use standard param_grid
        param_grid = cv_config.get('param_grid', {})
    
    return param_grid


def build_model_pipeline(model_name: str, task_type: str, 
                        model_params: dict, X: pd.DataFrame) -> Pipeline:
    """Build sklearn pipeline with preprocessing + model."""
    numeric_features, categorical_features = identify_feature_types(X)
    preprocessor = create_preprocessor(numeric_features, categorical_features)
    
    # Create model
    if task_type == 'classification':
        if model_name == 'logreg':
            model = LogisticRegression(**model_params)
        elif model_name == 'lgbm':
            model = lgb.LGBMClassifier(**model_params)
        else:
            raise ValueError(f"Unknown classifier: {model_name}")
    else:  # regression
        if model_name == 'ridge':
            model = Ridge(**model_params)
        elif model_name == 'lgbm_regressor':
            model = lgb.LGBMRegressor(**model_params)
        elif model_name == 'tweedie':
            model = TweedieRegressor(**model_params)
        else:
            raise ValueError(f"Unknown regressor: {model_name}")
    
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    return pipeline


def run_cv_grid_search(X: pd.DataFrame, y: pd.Series, 
                      model_name: str, task_type: str,
                      config: dict) -> Tuple[Pipeline, dict]:
    """
    Run CV grid search with adaptive parameter grids.
    
    Returns:
        best_pipeline: Fitted pipeline with best params
        cv_results: Dictionary with CV metadata
    """
    task_config = config.get(task_type, {})
    model_config = task_config.get(model_name, {})
    
    # Get base parameters
    base_params = model_config.get('base_params', {})
    
    # Get CV configuration
    cv_config = model_config.get('cv_search', {})
    
    if not cv_config.get('enabled', False):
        # No CV, just train with base params
        pipeline = build_model_pipeline(model_name, task_type, base_params, X)
        pipeline.fit(X, y)
        return pipeline, {'cv_enabled': False, 'best_params': base_params}
    
    # Calculate label characteristics
    if task_type == 'classification':
        label_sparsity = y.mean()
    else:
        label_sparsity = (y != 0).mean()
    
    # Get adaptive param grid
    param_grid = get_adaptive_param_grid(
        model_name, task_type, label_sparsity, len(X), config
    )
    
    if not param_grid:
        # No grid specified, use base params
        pipeline = build_model_pipeline(model_name, task_type, base_params, X)
        pipeline.fit(X, y)
        return pipeline, {'cv_enabled': False, 'best_params': base_params}
    
    # Prepend 'model__' to param names for pipeline
    param_grid_pipeline = {f'model__{k}': v for k, v in param_grid.items()}
    
    # Build base pipeline
    pipeline = build_model_pipeline(model_name, task_type, base_params, X)
    
    # Run grid search
    cv_folds = cv_config.get('cv_folds', 5)
    scoring = cv_config.get('scoring', 'roc_auc' if task_type == 'classification' else 'neg_mean_squared_error')
    
    # Calculate total possible combinations
    n_combinations = int(np.prod([len(v) for v in param_grid.values()]))
    
    # Get N_ITER from config (if specified)
    n_iter = cv_config.get('n_iter', None)
    
    # Use RandomizedSearchCV if n_iter specified and less than total combinations
    if n_iter and n_iter < n_combinations:
        search = RandomizedSearchCV(
            pipeline,
            param_grid_pipeline,
            n_iter=n_iter,
            cv=KFold(n_splits=cv_folds, shuffle=True, random_state=42),
            scoring=scoring,
            n_jobs=-1,
            verbose=0,
            random_state=42
        )
        
        if VERBOSE:
            print(f"    Running {cv_folds}-fold CV over {n_iter} random combinations (from {n_combinations} possible)...")
    else:
        search = GridSearchCV(
            pipeline,
            param_grid_pipeline,
            cv=KFold(n_splits=cv_folds, shuffle=True, random_state=42),
            scoring=scoring,
            n_jobs=-1,
            verbose=0
        )
        
        if VERBOSE:
            print(f"    Running {cv_folds}-fold CV over {n_combinations} parameter combinations...")
    
    search.fit(X, y)
    
    best_params_full = search.best_params_
    # Remove 'model__' prefix
    best_params = {k.replace('model__', ''): v for k, v in best_params_full.items()}
    
    cv_results = {
        'cv_enabled': True,
        'best_params': best_params,
        'best_score': search.best_score_,
        'cv_folds': cv_folds,
        'n_combinations': n_iter if n_iter else n_combinations
    }
    
    if VERBOSE:
        print(f"    Best CV score: {search.best_score_:.4f}")
        print(f"    Best params: {best_params}")
    
    return search.best_estimator_, cv_results


def calibrate_model(pipeline: Pipeline, X: pd.DataFrame, y: pd.Series,
                   config: dict, model_name: str, task_type: str) -> Pipeline:
    """Apply calibration to classification models."""
    if task_type != 'classification':
        return pipeline
    
    task_config = config.get(task_type, {})
    model_config = task_config.get(model_name, {})
    calib_config = model_config.get('calibration', {})
    
    if not calib_config.get('enabled', False):
        return pipeline
    
    method = calib_config.get('method', 'isotonic')
    
    if VERBOSE:
        print(f"    Applying {method} calibration...")
    
    calibrated = CalibratedClassifierCV(
        pipeline,
        method=method,
        cv='prefit',
        ensemble=False
    )
    
    calibrated.fit(X, y)
    
    return calibrated


print("✓ Model building functions defined")

✓ Model building functions defined


---
# Section 6: Ensemble Functions

In [7]:
# ============================================================================
# ENSEMBLE FUNCTIONS
# ============================================================================

class WeightedEnsemble:
    """Sklearn-compatible weighted ensemble."""
    
    def __init__(self, models: dict, weights: dict, task_type: str):
        self.models = models
        self.weights = weights
        self.task_type = task_type
    
    def predict_proba(self, X):
        """Weighted average of probabilities (classification only)."""
        if self.task_type != 'classification':
            raise ValueError("predict_proba only for classification")
        
        predictions = []
        for name, model in self.models.items():
            pred = model.predict_proba(X)
            predictions.append(pred * self.weights[name])
        
        return np.sum(predictions, axis=0)
    
    def predict(self, X):
        """Weighted average prediction."""
        if self.task_type == 'classification':
            proba = self.predict_proba(X)
            return (proba[:, 1] > 0.5).astype(int)
        else:
            predictions = []
            for name, model in self.models.items():
                pred = model.predict(X)
                predictions.append(pred * self.weights[name])
            return np.sum(predictions, axis=0)


def should_create_ensemble(model_performances: dict, config: dict) -> bool:
    """
    Decide if ensemble is worth creating based on performance similarity.
    Only create if models are "close" in performance.
    """
    ensemble_config = config.get('ensemble', {})
    
    if not ensemble_config.get('enabled', False):
        return False
    
    similarity_threshold = ensemble_config.get('similarity_threshold', 0.02)
    
    # Get performance metric for each model
    scores = [perf['primary_metric'] for perf in model_performances.values()]
    
    if len(scores) < 2:
        return False
    
    # Check if models are similar in performance
    score_range = max(scores) - min(scores)
    
    if score_range > similarity_threshold:
        if VERBOSE:
            print(f"  Ensemble skipped: performance gap ({score_range:.4f}) "
                  f"exceeds threshold ({similarity_threshold})")
        return False
    
    return True


def calculate_ensemble_weights(model_performances: dict, 
                              strategy: str = 'performance') -> dict:
    """
    Calculate ensemble weights based on strategy.
    
    Strategies:
    - 'equal': All models weighted equally
    - 'performance': Weight by relative performance
    """
    if strategy == 'equal':
        n_models = len(model_performances)
        return {name: 1.0 / n_models for name in model_performances.keys()}
    
    elif strategy == 'performance':
        # Weight by performance score
        scores = {name: perf['primary_metric'] 
                 for name, perf in model_performances.items()}
        
        # Normalize to sum to 1
        total_score = sum(scores.values())
        weights = {name: score / total_score 
                  for name, score in scores.items()}
        
        return weights
    
    else:
        raise ValueError(f"Unknown weighting strategy: {strategy}")


def create_ensemble(models: dict, model_performances: dict,
                   config: dict, task_type: str) -> Optional[WeightedEnsemble]:
    """
    Create weighted ensemble if appropriate.
    
    Returns:
        WeightedEnsemble or None if ensemble not created
    """
    if not should_create_ensemble(model_performances, config):
        return None
    
    ensemble_config = config.get('ensemble', {})
    weighting_config = ensemble_config.get('weighting_strategy', {})
    strategy = weighting_config.get('method', 'performance')
    
    weights = calculate_ensemble_weights(model_performances, strategy)
    
    if VERBOSE:
        print(f"\n  Creating ensemble with {strategy} weighting:")
        for name, weight in weights.items():
            print(f"    {name}: {weight:.3f}")
    
    ensemble = WeightedEnsemble(models, weights, task_type)
    
    return ensemble


print("✓ Ensemble functions defined")

✓ Ensemble functions defined


---
# Section 7: Evaluation Functions

In [8]:
# ============================================================================
# EVALUATION FUNCTIONS
# ============================================================================

def evaluate_classification(y_true: np.ndarray, y_pred_proba: np.ndarray,
                           config: dict) -> dict:
    """
    Calculate all classification metrics.
    
    Returns dict with:
    - core metrics (AUC, PR-AUC, Brier, etc.)
    - lift table
    - lift at K values
    """
    eval_config = config.get('classification', {})
    
    metrics = {}
    
    # Core metrics
    metrics['auc'] = roc_auc_score(y_true, y_pred_proba)
    metrics['pr_auc'] = average_precision_score(y_true, y_pred_proba)
    metrics['brier_score'] = brier_score_loss(y_true, y_pred_proba)
    metrics['log_loss'] = log_loss(y_true, y_pred_proba)
    metrics['baseline_rate'] = y_true.mean()
    
    # Lift table
    lift_config = eval_config.get('lift_table', {})
    n_quantiles = lift_config.get('n_quantiles', 20)
    lift_at_k = lift_config.get('lift_at_k', [0.01, 0.05, 0.10])
    
    df_lift = pd.DataFrame({
        'y_true': y_true,
        'y_pred_proba': y_pred_proba
    })
    
    df_lift = df_lift.sort_values('y_pred_proba', ascending=False).reset_index(drop=True)
    df_lift['quantile'] = pd.qcut(df_lift.index, n_quantiles, labels=False, duplicates='drop') + 1
    
    lift_table = df_lift.groupby('quantile').agg(
        n=('y_true', 'count'),
        n_events=('y_true', 'sum'),
        avg_score=('y_pred_proba', 'mean')
    ).reset_index()
    
    baseline = y_true.mean()
    lift_table['event_rate'] = lift_table['n_events'] / lift_table['n']
    lift_table['lift'] = lift_table['event_rate'] / baseline
    lift_table['cum_n'] = lift_table['n'].cumsum()
    lift_table['cum_n_events'] = lift_table['n_events'].cumsum()
    lift_table['cum_event_rate'] = lift_table['cum_n_events'] / lift_table['cum_n']
    lift_table['cum_lift'] = lift_table['cum_event_rate'] / baseline
    lift_table['cum_pct_captured'] = lift_table['cum_n_events'] / y_true.sum()
    
    metrics['lift_table'] = lift_table
    
    # Lift at K
    for k in lift_at_k:
        n_k = max(1, int(len(df_lift) * k))
        top_k_events = df_lift.head(n_k)['y_true'].sum()
        expected_k_events = baseline * n_k
        lift_k = top_k_events / expected_k_events if expected_k_events > 0 else 0
        metrics[f'lift_at_{k}'] = lift_k
    
    return metrics


def evaluate_regression(y_true: np.ndarray, y_pred: np.ndarray,
                       config: dict) -> dict:
    """
    Calculate all regression metrics.
    """
    metrics = {}
    
    metrics['rmse'] = np.sqrt(mean_squared_error(y_true, y_pred))
    metrics['mae'] = mean_absolute_error(y_true, y_pred)
    metrics['r2'] = r2_score(y_true, y_pred)
    
    # MAPE (avoid division by zero)
    mask = y_true != 0
    if mask.sum() > 0:
        metrics['mape'] = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))
    else:
        metrics['mape'] = np.nan
    
    # Baseline (predict mean)
    baseline_pred = np.full_like(y_true, y_true.mean())
    metrics['baseline_rmse'] = np.sqrt(mean_squared_error(y_true, baseline_pred))
    
    return metrics


def select_best_model(candidate_results: dict, task_type: str, config: dict) -> str:
    """
    Select best model based on configuration criteria.
    
    Returns:
        Name of best model
    """
    selection_config = config.get('best_model_selection', {})
    
    # Check for manual override
    manual = selection_config.get('manual_selection')
    if manual and manual in candidate_results:
        if VERBOSE:
            print(f"  Manual selection: {manual}")
        return manual
    
    # Get primary metric
    primary_metrics = selection_config.get('primary_metric', {})
    primary_metric = primary_metrics.get(task_type, 'oot_auc' if task_type == 'classification' else 'oot_rmse')
    
    # Check for overfitting penalty
    overfit_config = selection_config.get('overfit_penalty', {})
    penalize_overfit = overfit_config.get('enabled', True)
    
    if task_type == 'classification':
        max_gap = overfit_config.get('max_gap', {}).get('classification', 0.05)
        
        # Find model with best OOT performance and acceptable train/OOT gap
        valid_models = {}
        for name, results in candidate_results.items():
            train_metric = results['metrics']['train'][primary_metric.replace('oot_', '')]
            oot_metric = results['metrics']['oot'][primary_metric.replace('oot_', '')]
            gap = train_metric - oot_metric
            
            if not penalize_overfit or gap <= max_gap:
                valid_models[name] = oot_metric
        
        if not valid_models:
            if VERBOSE:
                print(f"  ⚠️  All models exceed overfit threshold, selecting best OOT anyway")
            valid_models = {name: results['metrics']['oot'][primary_metric.replace('oot_', '')]
                          for name, results in candidate_results.items()}
        
        # Maximize AUC
        best_model = max(valid_models, key=valid_models.get)
        
    else:  # regression
        max_gap_ratio = overfit_config.get('max_gap', {}).get('regression', 0.20)
        
        valid_models = {}
        for name, results in candidate_results.items():
            train_metric = results['metrics']['train']['rmse']
            oot_metric = results['metrics']['oot']['rmse']
            gap_ratio = (oot_metric - train_metric) / train_metric if train_metric > 0 else 999
            
            if not penalize_overfit or gap_ratio <= max_gap_ratio:
                valid_models[name] = oot_metric
        
        if not valid_models:
            if VERBOSE:
                print(f"  ⚠️  All models exceed overfit threshold, selecting best OOT anyway")
            valid_models = {name: results['metrics']['oot']['rmse']
                          for name, results in candidate_results.items()}
        
        # Minimize RMSE
        best_model = min(valid_models, key=valid_models.get)
    
    if VERBOSE:
        print(f"  Best model: {best_model} ({primary_metric}={valid_models[best_model]:.4f})")
    
    return best_model


print("✓ Evaluation functions defined")

✓ Evaluation functions defined


---
# Section 8: Threshold Optimization

In [9]:
# ============================================================================
# THRESHOLD OPTIMIZATION FUNCTIONS
# ============================================================================

def optimize_thresholds(y_true: np.ndarray, y_pred_proba: np.ndarray,
                       config: dict) -> dict:
    """
    Find optimal thresholds for different business objectives.
    
    Returns dict with threshold for each objective.
    """
    threshold_config = config.get('threshold_optimization', {})
    
    if not threshold_config.get('enabled', False):
        return {}
    
    search_config = threshold_config.get('search', {})
    n_thresholds = search_config.get('n_thresholds', 100)
    min_threshold = search_config.get('min_threshold', 0.01)
    max_threshold = search_config.get('max_threshold', 0.99)
    
    thresholds_to_test = np.linspace(min_threshold, max_threshold, n_thresholds)
    
    results = {}
    objectives_config = threshold_config.get('objectives', {})
    
    # Calculate metrics at each threshold
    threshold_metrics = []
    for thresh in thresholds_to_test:
        y_pred = (y_pred_proba >= thresh).astype(int)
        
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        
        threshold_metrics.append({
            'threshold': thresh,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
    
    df_metrics = pd.DataFrame(threshold_metrics)
    
    # Find optimal threshold for each objective
    for obj_name, obj_config in objectives_config.items():
        metric = obj_config.get('metric')
        optimization = obj_config.get('optimization')
        
        if obj_name == 'profit_optimal' and not obj_config.get('enabled', False):
            continue
        
        if optimization == 'maximize':
            if metric == 'profit':
                cost_per_contact = obj_config.get('cost_per_contact', 2.0)
                value_per_conversion = obj_config.get('value_per_conversion', 50.0)
                
                profits = []
                for _, row in df_metrics.iterrows():
                    thresh = row['threshold']
                    y_pred = (y_pred_proba >= thresh).astype(int)
                    
                    n_contacts = y_pred.sum()
                    n_conversions = (y_true & y_pred).sum()
                    
                    profit = (n_conversions * value_per_conversion) - (n_contacts * cost_per_contact)
                    profits.append(profit)
                
                df_metrics['profit'] = profits
                best_idx = df_metrics['profit'].idxmax()
                results[obj_name] = {
                    'threshold': df_metrics.loc[best_idx, 'threshold'],
                    'profit': df_metrics.loc[best_idx, 'profit']
                }
            else:
                best_idx = df_metrics[metric].idxmax()
                results[obj_name] = {
                    'threshold': df_metrics.loc[best_idx, 'threshold'],
                    metric: df_metrics.loc[best_idx, metric]
                }
        
        elif optimization == 'reach_target':
            target_value = obj_config.get('target_value')
            # Find threshold closest to target
            df_metrics['distance'] = np.abs(df_metrics[metric] - target_value)
            best_idx = df_metrics['distance'].idxmin()
            results[obj_name] = {
                'threshold': df_metrics.loc[best_idx, 'threshold'],
                metric: df_metrics.loc[best_idx, metric],
                'target': target_value
            }
    
    # Store full threshold curve for plotting
    results['_threshold_curve'] = df_metrics
    
    if VERBOSE:
        print(f"\n  Threshold Optimization:")
        for obj_name, obj_result in results.items():
            if obj_name != '_threshold_curve':
                print(f"    {obj_name}: threshold={obj_result['threshold']:.3f}")
    
    return results


print("✓ Threshold optimization functions defined")

✓ Threshold optimization functions defined


---
# Section 9: SHAP / Explainability Functions

In [10]:
# ============================================================================
# SHAP / EXPLAINABILITY FUNCTIONS
# ============================================================================

def generate_shap_values(base_model, X, eval_config=None):
    """
    Generate SHAP values for tree-based models only.
    For linear models (LogisticRegression), we skip SHAP and rely on
    coefficient / odds-ratio tables instead.

    Parameters
    ----------
    base_model : estimator or sklearn.Pipeline
        Fitted model or pipeline. If a Pipeline, we expect a 'preprocessor'
        and 'model' step; otherwise we treat it as a plain estimator.
    X : pd.DataFrame
        Original feature data (already feature-selected in your pipeline).
    eval_config : dict, optional
        Can carry SHAP-related settings, e.g.:
          - 'shap_sample_size': int or None
          - 'random_state': int

    Returns
    -------
    shap_values : array-like or None
        SHAP values for the sampled data, or None if SHAP is skipped/failed.
    """
    # --- Pull options from eval_config ---
    sample_size = None
    random_state = 42
    if isinstance(eval_config, dict):
        sample_size = eval_config.get("shap_sample_size", sample_size)
        random_state = eval_config.get("random_state", random_state)

    # --- Subsample X if requested ---
    if sample_size is not None and len(X) > sample_size:
        X_sample = X.sample(n=sample_size, random_state=random_state)
    else:
        X_sample = X

    # --- Transform with preprocessor if present (Pipeline case) ---
    preprocessor = None
    if hasattr(base_model, "named_steps"):
        preprocessor = base_model.named_steps.get("preprocessor", None)

    if preprocessor is not None:
        X_transformed = preprocessor.transform(X_sample)
    else:
        # assume X is already numeric or compatible with the model
        X_transformed = X_sample.values

    # --- Get underlying estimator ---
    if hasattr(base_model, "named_steps") and "model" in base_model.named_steps:
        actual_model = base_model.named_steps["model"]
    else:
        actual_model = base_model

    # --- If it's logistic, skip SHAP (we'll use coeff table instead) ---
    if isinstance(actual_model, LogisticRegression):
        print("Skipping SHAP for LogisticRegression – using coefficient/odds table instead.")
        return None

    # --- Otherwise, run SHAP (tree or other) ---
    try:
        explainer = shap.Explainer(actual_model, X_transformed)

        # For TreeExplainer, disable strict additivity check
        if isinstance(explainer, shap.explainers._tree.TreeExplainer):
            shap_values = explainer(X_transformed, check_additivity=False)
        else:
            shap_values = explainer(X_transformed)

        return shap_values

    except Exception as e:
        print(f"  ⚠️  SHAP generation failed: {e}")
        return None


def summarize_shap_direction(shap_dict: dict, top_n: int = 20) -> pd.DataFrame:
    """
    Summarize SHAP directionality (positive vs negative impact).
    
    Returns DataFrame with:
    - feature: Feature name
    - mean_abs_shap: Mean absolute SHAP value
    - corr_feature_shap: Correlation between feature value and SHAP value
    - direction: Interpretation of direction
    """
    if shap_dict is None:
        return None
    
    shap_values = shap_dict['values']
    X_data = shap_dict['data']
    feature_names = shap_dict['feature_names']
    
    mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
    order = np.argsort(mean_abs_shap)[::-1][:top_n]
    
    rows = []
    for idx in order:
        fname = feature_names[idx]
        vals = X_data[:, idx]
        shap_col = shap_values[:, idx]
        
        # Drop NaNs if any
        mask = ~np.isnan(vals) & ~np.isnan(shap_col)
        vals = vals[mask]
        shap_col = shap_col[mask]
        
        if len(vals) < 10:
            corr = np.nan
            sign = "unknown"
        else:
            corr = np.corrcoef(vals, shap_col)[0, 1]
            if corr > 0.05:
                sign = "higher value → higher prediction"
            elif corr < -0.05:
                sign = "higher value → lower prediction"
            else:
                sign = "mixed / weak"
        
        rows.append({
            "feature": fname,
            "mean_abs_shap": mean_abs_shap[idx],
            "corr_feature_shap": corr,
            "direction": sign
        })
    
    return pd.DataFrame(rows)


def build_logistic_coeff_table(
    base_model,
    feature_names=None,
    top_n=None,
    round_digits=3,
):
    """
    Build an interpretability table for a logistic regression model:
    - coefficient (log-odds)
    - odds ratio (exp(coefficient))
    - absolute coefficient for ranking
    - plain-English explanation of odds ratio

    Parameters
    ----------
    base_model : sklearn.Pipeline
        Fitted pipeline with a 'model' step that is a LogisticRegression.
    feature_names : list of str, optional
        Names of the features after preprocessing. If None, will try
        base_model.named_steps['preprocessor'].get_feature_names_out().
    top_n : int, optional
        If provided, return only the top_n features by |coefficient|.
    round_digits : int, optional
        Number of decimal places for rounding numeric columns.

    Returns
    -------
    coeff_df : pandas.DataFrame
        Columns:
        ['feature', 'coefficient', 'odds_ratio',
         'abs_coefficient', 'interpretation']
        Sorted by |coefficient| descending.
    """
    # 1. Get the underlying model
    model = base_model.named_steps.get("model", None)
    if model is None:
        raise ValueError("Pipeline must have a 'model' step.")
    if not isinstance(model, LogisticRegression):
        raise ValueError(
            f"'model' step must be LogisticRegression, got {type(model)} instead."
        )

    # 2. Resolve feature names
    if feature_names is None:
        preprocessor = base_model.named_steps.get("preprocessor", None)
        if preprocessor is not None and hasattr(preprocessor, "get_feature_names_out"):
            feature_names = preprocessor.get_feature_names_out()
        else:
            n_features = model.coef_.shape[1]
            feature_names = [f"feature_{i}" for i in range(n_features)]

    # 3. Extract coefficients (binary classification: (1, n_features))
    coef = model.coef_.ravel()
    odds_ratio = np.exp(coef)

    coeff_df = pd.DataFrame(
        {
            "feature": feature_names,
            "coefficient": coef,
            "odds_ratio": odds_ratio,
        }
    )

    # 4. Add abs(coef) for ranking
    coeff_df["abs_coefficient"] = coeff_df["coefficient"].abs()

    # 5. Sort by importance
    coeff_df = coeff_df.sort_values("abs_coefficient", ascending=False)

    # 6. Keep only top_n if requested
    if top_n is not None:
        coeff_df = coeff_df.head(top_n)

    # 7. Round numeric columns for readability
    numeric_cols = ["coefficient", "odds_ratio", "abs_coefficient"]
    coeff_df[numeric_cols] = coeff_df[numeric_cols].round(round_digits)

    # 8. Build plain-English interpretation column
    interpretations = []
    for _, row in coeff_df.iterrows():
        feat = row["feature"]
        or_val = row["odds_ratio"]

        # percent change in odds for a 1-unit increase
        pct_change = (or_val - 1.0) * 100.0
        pct_change_rounded = round(pct_change, 1)

        if abs(pct_change_rounded) < 0.5:
            text = (
                f"A 1-unit increase in '{feat}' is associated with "
                f"almost no change in the odds of the positive outcome."
            )
        elif pct_change_rounded > 0:
            text = (
                f"A 1-unit increase in '{feat}' is associated with "
                f"approximately {pct_change_rounded}% higher odds of the "
                f"positive outcome"
            )
        else:
            text = (
                f"A 1-unit increase in '{feat}' is associated with "
                f"approximately {abs(pct_change_rounded)}% lower odds of the "
                f"positive outcome"
            )

        interpretations.append(text)

    coeff_df["interpretation"] = interpretations

    return coeff_df


print("✓ SHAP functions defined")

✓ SHAP functions defined


---
# Section 10: Visualization Functions

In [11]:
# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def plot_lift_curve(lift_table: pd.DataFrame, output_path: str):
    """Plot lift curve from lift table."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Cumulative lift
    ax1.plot(lift_table['quantile'], lift_table['cum_lift'], marker='o', linewidth=2)
    ax1.axhline(y=1, color='red', linestyle='--', label='Baseline')
    ax1.set_xlabel('Quantile')
    ax1.set_ylabel('Cumulative Lift')
    ax1.set_title('Cumulative Lift')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Cumulative % captured
    ax2.plot(lift_table['quantile'], lift_table['cum_pct_captured'] * 100, 
             marker='o', linewidth=2)
    ax2.set_xlabel('Quantile')
    ax2.set_ylabel('% Events Captured')
    ax2.set_title('Event Capture')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()


def plot_calibration_curve(y_true: np.ndarray, y_pred_proba: np.ndarray,
                          output_path: str, n_bins: int = 10):
    """Plot calibration curve."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Calculate calibration
    bins = np.linspace(0, 1, n_bins + 1)
    bin_indices = np.digitize(y_pred_proba, bins) - 1
    bin_indices = np.clip(bin_indices, 0, n_bins - 1)
    
    bin_true = np.zeros(n_bins)
    bin_pred = np.zeros(n_bins)
    bin_count = np.zeros(n_bins)
    
    for i in range(n_bins):
        mask = bin_indices == i
        if mask.sum() > 0:
            bin_true[i] = y_true[mask].mean()
            bin_pred[i] = y_pred_proba[mask].mean()
            bin_count[i] = mask.sum()
    
    # Calibration curve
    ax1.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
    ax1.plot(bin_pred, bin_true, marker='o', linewidth=2, label='Model')
    ax1.set_xlabel('Predicted Probability')
    ax1.set_ylabel('Actual Probability')
    ax1.set_title('Calibration Curve')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Distribution of predictions
    ax2.hist(y_pred_proba, bins=50, alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Predicted Probability')
    ax2.set_ylabel('Count')
    ax2.set_title('Distribution of Predictions')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()


def plot_shap_summary(shap_dict: dict, output_path: str, top_n: int = 20):
    """Plot SHAP summary."""
    if shap_dict is None:
        print("  ⚠️  No SHAP data to plot")
        return
    
    try:
        shap_values = shap_dict['values']
        X_data = shap_dict['data']
        feature_names = shap_dict['feature_names']
        
        plt.figure(figsize=(10, max(6, top_n * 0.3)))
        shap.summary_plot(
            shap_values,
            X_data,
            feature_names=feature_names,
            max_display=top_n,
            show=False
        )
        plt.tight_layout()
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
        plt.close()
    except Exception as e:
        print(f"  ⚠️  SHAP plot failed: {e}")


def plot_threshold_analysis(threshold_results: dict, output_path: str):
    """Plot precision/recall/F1 vs threshold."""
    if '_threshold_curve' not in threshold_results:
        return
    
    df = threshold_results['_threshold_curve']
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(df['threshold'], df['precision'], label='Precision', linewidth=2)
    ax.plot(df['threshold'], df['recall'], label='Recall', linewidth=2)
    ax.plot(df['threshold'], df['f1'], label='F1', linewidth=2)
    
    # Mark optimal thresholds
    for obj_name, obj_result in threshold_results.items():
        if obj_name != '_threshold_curve':
            thresh = obj_result['threshold']
            ax.axvline(x=thresh, color='red', linestyle='--', alpha=0.5)
            ax.text(thresh, 0.95, obj_name, rotation=90, 
                   verticalalignment='top', fontsize=8)
    
    ax.set_xlabel('Threshold')
    ax.set_ylabel('Score')
    ax.set_title('Metrics vs Threshold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()


print("✓ Visualization functions defined")

✓ Visualization functions defined


---
# Section 11: Main Pipeline Orchestrator

In [12]:
# ============================================================================
# MAIN PIPELINE ORCHESTRATOR
# ============================================================================

def train_single_cohort_model(cohort_name: str, label: str, 
                             train_data: pd.DataFrame, oot_data: pd.DataFrame,
                             model_names: List[str], task_type: str) -> dict:
    """
    Train models for a single cohort.
    
    Returns dict with:
    - candidate_results: Results for all candidate models
    - best_model_name: Name of best model
    - best_model: Best model object
    - best_metrics: Metrics for best model
    - feature_selection_info: Feature selection metadata
    - shap_values: SHAP values for best model (for tree-based models)
    - coeff_table: Coefficient / odds-ratio table (for logistic models)
    - threshold_results: Threshold optimization results
    """
    if VERBOSE:
        print(f"\n{'='*80}")
        print(f"Cohort: {cohort_name}")
        print(f"{'='*80}")
    
    # Split features and labels
    X_train, y_train, train_ids = split_features_labels(train_data, label)
    X_oot, y_oot, oot_ids = split_features_labels(oot_data, label)
    
    if VERBOSE:
        print(f"\nData sizes:")
        print(f"  Train: {len(X_train):,} samples, {X_train.shape[1]} features")
        print(f"  OOT: {len(X_oot):,} samples")
        print(f"  Label prevalence (train): {y_train.mean():.2%}")
    
    # Feature selection
    X_train_selected, feature_selection_info = run_feature_selection(
        X_train, y_train, FEATURE_CONFIG
    )
    X_oot_selected = X_oot[feature_selection_info['feature_names']]
    
    # Train candidate models
    candidate_results = {}
    
    for model_name in model_names:
        if VERBOSE:
            print(f"\n{'─'*60}")
            print(f"Training: {model_name}")
            print(f"{'─'*60}")
        
        try:
            # Train with CV grid search
            model, cv_results = run_cv_grid_search(
                X_train_selected, y_train, model_name, task_type, MODEL_CONFIG
            )
            
            # Calibrate if classification
            if task_type == 'classification':
                model = calibrate_model(
                    model, X_train_selected, y_train, 
                    MODEL_CONFIG, model_name, task_type
                )
            
            # Get predictions
            if task_type == 'classification':
                train_pred = model.predict_proba(X_train_selected)[:, 1]
                oot_pred = model.predict_proba(X_oot_selected)[:, 1]
            else:
                train_pred = model.predict(X_train_selected)
                oot_pred = model.predict(X_oot_selected)
            
            # Evaluate
            if task_type == 'classification':
                train_metrics = evaluate_classification(
                    y_train.values, train_pred, EVAL_CONFIG
                )
                oot_metrics = evaluate_classification(
                    y_oot.values, oot_pred, EVAL_CONFIG
                )
                
                if VERBOSE:
                    print(f"  Train AUC: {train_metrics['auc']:.4f} | "
                          f"PR-AUC: {train_metrics['pr_auc']:.4f}")
                    print(f"  OOT   AUC: {oot_metrics['auc']:.4f} | "
                          f"PR-AUC: {oot_metrics['pr_auc']:.4f}")
            else:
                train_metrics = evaluate_regression(
                    y_train.values, train_pred, EVAL_CONFIG
                )
                oot_metrics = evaluate_regression(
                    y_oot.values, oot_pred, EVAL_CONFIG
                )
                
                if VERBOSE:
                    print(f"  Train RMSE: {train_metrics['rmse']:.4f} | "
                          f"R²: {train_metrics['r2']:.4f}")
                    print(f"  OOT   RMSE: {oot_metrics['rmse']:.4f} | "
                          f"R²: {oot_metrics['r2']:.4f}")
            
            # Store results
            candidate_results[model_name] = {
                'model': model,
                'cv_results': cv_results,
                'metrics': {
                    'train': train_metrics,
                    'oot': oot_metrics
                },
                'predictions': {
                    'train': pd.DataFrame({
                        'donor_id': train_ids,
                        'actual': y_train,
                        'predicted': train_pred
                    }),
                    'oot': pd.DataFrame({
                        'donor_id': oot_ids,
                        'actual': y_oot,
                        'predicted': oot_pred
                    })
                }
            }
            
        except Exception as e:
            print(f"  ⚠️  {model_name} failed: {e}")
            continue
    
    if not candidate_results:
        raise RuntimeError("All models failed to train")
    
    # Optional: Create ensemble
    if ENABLE_ENSEMBLE and len(candidate_results) > 1:
        if VERBOSE:
            print(f"\n{'─'*60}")
            print(f"Evaluating ensemble")
            print(f"{'─'*60}")
        
        # Prepare model performances for ensemble decision
        model_performances = {}
        for name, results in candidate_results.items():
            if task_type == 'classification':
                primary_metric = results['metrics']['oot']['auc']
            else:
                primary_metric = results['metrics']['oot']['rmse']
            
            model_performances[name] = {
                'primary_metric': primary_metric
            }
        
        # Create ensemble if appropriate
        ensemble_config_full = MODEL_CONFIG.copy()
        ensemble_config_full['ensemble'] = {'enabled': True, 'weighting_strategy': {'method': 'performance'}, 'similarity_threshold': 0.02}
        
        ensemble_model = create_ensemble(
            {name: res['model'] for name, res in candidate_results.items()},
            model_performances,
            ensemble_config_full,
            task_type
        )
        
        if ensemble_model is not None:
            # Evaluate ensemble
            if task_type == 'classification':
                train_pred = ensemble_model.predict_proba(X_train_selected)[:, 1]
                oot_pred = ensemble_model.predict_proba(X_oot_selected)[:, 1]
            else:
                train_pred = ensemble_model.predict(X_train_selected)
                oot_pred = ensemble_model.predict(X_oot_selected)
            
            if task_type == 'classification':
                train_metrics = evaluate_classification(
                    y_train.values, train_pred, EVAL_CONFIG
                )
                oot_metrics = evaluate_classification(
                    y_oot.values, oot_pred, EVAL_CONFIG
                )
                
                if VERBOSE:
                    print(f"  Ensemble Train AUC: {train_metrics['auc']:.4f}")
                    print(f"  Ensemble OOT   AUC: {oot_metrics['auc']:.4f}")
            else:
                train_metrics = evaluate_regression(
                    y_train.values, train_pred, EVAL_CONFIG
                )
                oot_metrics = evaluate_regression(
                    y_oot.values, oot_pred, EVAL_CONFIG
                )
                
                if VERBOSE:
                    print(f"  Ensemble Train RMSE: {train_metrics['rmse']:.4f}")
                    print(f"  Ensemble OOT   RMSE: {oot_metrics['rmse']:.4f}")
            
            # Add ensemble to candidates
            candidate_results['ensemble'] = {
                'model': ensemble_model,
                'cv_results': {'cv_enabled': False},
                'metrics': {
                    'train': train_metrics,
                    'oot': oot_metrics
                },
                'predictions': {
                    'train': pd.DataFrame({
                        'donor_id': train_ids,
                        'actual': y_train,
                        'predicted': train_pred
                    }),
                    'oot': pd.DataFrame({
                        'donor_id': oot_ids,
                        'actual': y_oot,
                        'predicted': oot_pred
                    })
                }
            }
    
    # Select best model
    if VERBOSE:
        print(f"\n{'─'*60}")
        print(f"Selecting best model")
        print(f"{'─'*60}")
    
    best_model_name = select_best_model(candidate_results, task_type, MODEL_CONFIG)
    best_results = candidate_results[best_model_name]
    
    # Interpretability: SHAP for trees, odds table for logistic
    shap_values = None
    coeff_table = None
    
    if best_model_name != 'ensemble':
        # Start from the stored model (may be a CalibratedClassifierCV or a Pipeline)
        model_obj = best_results['model']
        
        # If calibrated, unwrap to the base_estimator (usually a Pipeline)
        if isinstance(model_obj, CalibratedClassifierCV):
            base_estimator = model_obj.estimator
        else:
            base_estimator = model_obj
        
        # If it's a Pipeline, pull out the final estimator
        if hasattr(base_estimator, "named_steps") and "model" in base_estimator.named_steps:
            actual_model = base_estimator.named_steps["model"]
        else:
            actual_model = base_estimator
        
        # LogisticRegression: use coefficients / odds ratios instead of SHAP
        if isinstance(actual_model, LogisticRegression) and task_type == 'classification':
            if VERBOSE:
                print("Using logistic regression coefficients / odds ratios instead of SHAP.")
            coeff_table = build_logistic_coeff_table(
                base_estimator,   # pipeline with preprocessor + model
                top_n=30,
                round_digits=3,
            )
        else:
            # Tree or other non-linear model: compute SHAP on base_estimator
            shap_values = generate_shap_values(
                base_estimator, X_train_selected, EVAL_CONFIG
            )
    
    # Optimize thresholds (classification only)
    threshold_results = {}
    if task_type == 'classification':
        threshold_results = optimize_thresholds(
            y_oot.values,
            best_results['predictions']['oot']['predicted'].values,
            EVAL_CONFIG
        )
    
    return {
        'candidate_results': candidate_results,
        'best_model_name': best_model_name,
        'best_model': best_results['model'],
        'best_metrics': best_results['metrics'],
        'best_predictions': best_results['predictions'],
        'feature_selection_info': feature_selection_info,
        'shap_values': shap_values,
        'coeff_table': coeff_table,
        'threshold_results': threshold_results,
        'X_train': X_train_selected,
        'X_oot': X_oot_selected,
        'y_train': y_train,
        'y_oot': y_oot
    }


print("✓ Main pipeline orchestrator defined")

✓ Main pipeline orchestrator defined


---
# Section 12: Execute Pipeline

In [13]:
# ============================================================================
# EXECUTE PIPELINE
# ============================================================================

# Load data
print(f"\n{'='*80}")
print(f"DONOR MODELING PIPELINE")
print(f"{'='*80}")
print(f"\nLoading data...")

train_features, oot_features = load_data(TRAIN_FEATURES_PATH, OOT_FEATURES_PATH)

print(f"✓ Data loaded")
print(f"  Train: {train_features.shape}")
print(f"  OOT: {oot_features.shape}")

# Validate label
if LABEL not in train_features.columns:
    available_labels = sorted([c for c in train_features.columns if c.startswith('label_')])
    print(f"\n⚠️  Label '{LABEL}' not found!")
    print(f"\nAvailable labels:")
    for lbl in available_labels[:10]:
        print(f"  - {lbl}")
    if len(available_labels) > 10:
        print(f"  ... and {len(available_labels) - 10} more")
    raise ValueError(f"Label '{LABEL}' not found")

# Detect task type
task_type = detect_task_type(train_features[LABEL])
print(f"\nLabel: {LABEL}")
print(f"Task type: {task_type}")

# Determine which cohorts to run
if COHORTS_TO_RUN is None:
    cohorts_to_run = list(COHORT_CONFIG['cohorts'].keys())
else:
    cohorts_to_run = COHORTS_TO_RUN

# Validate cohorts
for cohort in cohorts_to_run:
    if cohort not in COHORT_CONFIG['cohorts']:
        raise ValueError(f"Cohort '{cohort}' not found in config")

print(f"\nCohorts to run: {', '.join(cohorts_to_run)}")

# Determine which models to run
if MODELS_TO_RUN is None:
    if task_type == 'classification':
        models_to_run = [name for name, config in MODEL_CONFIG.get('classification', {}).items()
                        if config.get('enabled', False)]
    else:
        models_to_run = [name for name, config in MODEL_CONFIG.get('regression', {}).items()
                        if config.get('enabled', False)]
else:
    models_to_run = MODELS_TO_RUN

print(f"Models to run: {', '.join(models_to_run)}")

# Run pipeline for each cohort
all_cohort_results = {}

for cohort_name in cohorts_to_run:
    # Filter to cohort
    train_cohort = apply_cohort_filter(train_features, cohort_name, COHORT_CONFIG)
    oot_cohort = apply_cohort_filter(oot_features, cohort_name, COHORT_CONFIG)
    
    # Train models
    cohort_results = train_single_cohort_model(
        cohort_name, LABEL, train_cohort, oot_cohort,
        models_to_run, task_type
    )
    
    all_cohort_results[cohort_name] = cohort_results
    
    # Save results
    output_dir = Path(OUTPUT_BASE_DIR) / LABEL / cohort_name / 'best_model'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    plots_dir = output_dir / 'plots'
    plots_dir.mkdir(exist_ok=True)
    
    # Save model
    with open(output_dir / 'model.pkl', 'wb') as f:
        pickle.dump(cohort_results['best_model'], f)
    
    # Save config
    config_info = {
        'label': LABEL,
        'cohort': cohort_name,
        'task_type': task_type,
        'best_model_name': cohort_results['best_model_name'],
        'models_evaluated': list(cohort_results['candidate_results'].keys()),
        'n_features': len(cohort_results['feature_selection_info']['feature_names']),
        'feature_names': cohort_results['feature_selection_info']['feature_names'],
        'timestamp': datetime.now().isoformat()
    }
    with open(output_dir / 'config.json', 'w') as f:
        json.dump(config_info, f, indent=2)
    
    # Save metrics
    metrics_flat = {}
    for split in ['train', 'oot']:
        for metric, value in cohort_results['best_metrics'][split].items():
            if metric != 'lift_table':
                metrics_flat[f'{split}_{metric}'] = value
    
    with open(output_dir / 'metrics.json', 'w') as f:
        json.dump(metrics_flat, f, indent=2)
    
    # Save predictions
    cohort_results['best_predictions']['train'].to_csv(
        output_dir / 'predictions_train.csv', index=False
    )
    cohort_results['best_predictions']['oot'].to_csv(
        output_dir / 'predictions_oot.csv', index=False
    )
    
    # Save SHAP values
    if cohort_results['shap_values'] is not None:
        shap_df = pd.DataFrame(
            cohort_results['shap_values']['values'],
            columns=cohort_results['shap_values']['feature_names']
        )
        shap_df.to_csv(output_dir / 'shap_values.csv', index=False)
        
        # SHAP direction summary
        shap_direction = summarize_shap_direction(cohort_results['shap_values'])
        if shap_direction is not None:
            shap_direction.to_csv(output_dir / 'shap_direction_summary.csv', index=False)

    # Save coefficient table (for logistic regression)
    if cohort_results['coeff_table'] is not None:
        cohort_results['coeff_table'].to_csv(
            output_dir / 'logistic_coefficients.csv', 
            index=False
        )
        print(f"  ✓ Saved logistic regression coefficients table")
    
    # Save threshold results
    if cohort_results['threshold_results']:
        threshold_save = {k: v for k, v in cohort_results['threshold_results'].items()
                         if k != '_threshold_curve'}
        with open(output_dir / 'threshold_analysis.json', 'w') as f:
            json.dump(threshold_save, f, indent=2)
    
    # Generate plots
    if task_type == 'classification':
        # Lift curve
        lift_table = cohort_results['best_metrics']['oot']['lift_table']
        plot_lift_curve(lift_table, plots_dir / 'lift_curve.png')
        
        # Calibration
        plot_calibration_curve(
            cohort_results['y_oot'].values,
            cohort_results['best_predictions']['oot']['predicted'].values,
            plots_dir / 'calibration.png'
        )
        
        # Threshold analysis
        if cohort_results['threshold_results']:
            plot_threshold_analysis(
                cohort_results['threshold_results'],
                plots_dir / 'threshold_analysis.png'
            )
    
    # SHAP plot
    if cohort_results['shap_values'] is not None:
        plot_shap_summary(
            cohort_results['shap_values'],
            plots_dir / 'shap_summary.png',
            top_n=EVAL_CONFIG.get('shap', {}).get('top_features', 20)
        )
    
    # Save candidate models if requested
    if SAVE_CANDIDATE_MODELS:
        candidates_dir = Path(OUTPUT_BASE_DIR) / LABEL / cohort_name / 'candidates'
        for model_name, results in cohort_results['candidate_results'].items():
            model_dir = candidates_dir / model_name
            model_dir.mkdir(parents=True, exist_ok=True)
            
            with open(model_dir / 'model.pkl', 'wb') as f:
                pickle.dump(results['model'], f)
    
    print(f"\n✓ Results saved to: {output_dir}")

print(f"\n{'='*80}")
print(f"PIPELINE COMPLETE")
print(f"{'='*80}")
print(f"\nResults saved to: {OUTPUT_BASE_DIR}/{LABEL}/")


DONOR MODELING PIPELINE

Loading data...
✓ Data loaded
  Train: (662474, 219)
  OOT: (1037432, 230)

Label: label_will_give_during_giving_tuesday
Task type: classification

Cohorts to run: all_donors
Models to run: logreg, lgbm
  Cohort 'all_donors': 658,595 donors (99.4%)
  Cohort 'all_donors': 1,036,822 donors (99.9%)

Cohort: all_donors
  Features: 154 columns
  Labels: 658595 samples
  Excluded: 65 columns (53 label columns)
  Features: 165 columns
  Labels: 1036822 samples
  Excluded: 65 columns (53 label columns)

Data sizes:
  Train: 658,595 samples, 154 features
  OOT: 1,036,822 samples
  Label prevalence (train): 4.03%

Feature Selection Pipeline:
  Starting features: 154
  Removed 16 correlated features (>0.95)
  Selected 58 features via tree importance
  Top 5 features: email_open_rate_3m, days_since_last_gift, teacher_still_available_during_range, email_open_rate_velocity_3m_vs_12m, pct_amount_green_lifetime
  Removed 2 low-variance features (<0.01)
  Final features: 56

─

---
# Section 13: Results Summary & Review

In [14]:
# ============================================================================
# RESULTS SUMMARY
# ============================================================================

# Create summary table
summary_rows = []

for cohort_name, results in all_cohort_results.items():
    row = {
        'cohort': cohort_name,
        'best_model': results['best_model_name'],
        'n_features': len(results['feature_selection_info']['feature_names'])
    }
    
    # Add metrics
    if task_type == 'classification':
        row['train_auc'] = results['best_metrics']['train']['auc']
        row['oot_auc'] = results['best_metrics']['oot']['auc']
        row['train_pr_auc'] = results['best_metrics']['train']['pr_auc']
        row['oot_pr_auc'] = results['best_metrics']['oot']['pr_auc']
        row['oot_brier'] = results['best_metrics']['oot']['brier_score']
        
        # Add lift@K
        for k in [0.01, 0.05, 0.10]:
            key = f'lift_at_{k}'
            if key in results['best_metrics']['oot']:
                row[f'lift_at_{int(k*100)}pct'] = results['best_metrics']['oot'][key]
    else:
        row['train_rmse'] = results['best_metrics']['train']['rmse']
        row['oot_rmse'] = results['best_metrics']['oot']['rmse']
        row['train_r2'] = results['best_metrics']['train']['r2']
        row['oot_r2'] = results['best_metrics']['oot']['r2']
        row['oot_mae'] = results['best_metrics']['oot']['mae']
    
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)

print(f"\n{'='*80}")
print(f"RESULTS SUMMARY")
print(f"{'='*80}")
print(f"\nLabel: {LABEL}")
print(f"Task: {task_type}")
print(f"\n{summary_df.to_string(index=False)}")

# Show top features for each cohort
print(f"\n{'='*80}")
print(f"TOP FEATURES BY COHORT")
print(f"{'='*80}")

for cohort_name, results in all_cohort_results.items():
    print(f"\n{cohort_name}:")
    
    if results['shap_values'] is not None:
        shap_direction = summarize_shap_direction(results['shap_values'], top_n=10)
        if shap_direction is not None:
            print(shap_direction[['feature', 'mean_abs_shap', 'direction']].to_string(index=False))
    else:
        importance_df = results['feature_selection_info']['importance_scores']
        if importance_df is not None:
            print(importance_df.head(10)[['feature', 'importance']].to_string(index=False))

# Show threshold recommendations (classification only)
if task_type == 'classification':
    print(f"\n{'='*80}")
    print(f"THRESHOLD RECOMMENDATIONS")
    print(f"{'='*80}")
    
    for cohort_name, results in all_cohort_results.items():
        if results['threshold_results']:
            print(f"\n{cohort_name}:")
            for obj_name, obj_result in results['threshold_results'].items():
                if obj_name != '_threshold_curve':
                    print(f"  {obj_name}: {obj_result['threshold']:.3f}")

print(f"\n{'='*80}")
print(f"Pipeline execution complete!")
print(f"Results directory: {OUTPUT_BASE_DIR}/{LABEL}/")
print(f"{'='*80}")


# ============================================================================
# DETAILED RESULTS EXAMINATION
# ============================================================================

print(f"\n{'='*80}")
print(f"DETAILED RESULTS EXAMINATION")
print(f"{'='*80}")

# Select cohort to examine
COHORT_TO_EXAMINE = 'all_donors'  # Change this to examine different cohorts

if COHORT_TO_EXAMINE not in all_cohort_results:
    print(f"⚠️  Cohort '{COHORT_TO_EXAMINE}' not found")
    print(f"Available cohorts: {list(all_cohort_results.keys())}")
else:
    cohort_results = all_cohort_results[COHORT_TO_EXAMINE]
    
    print(f"\n{'─'*80}")
    print(f"COHORT: {COHORT_TO_EXAMINE}")
    print(f"{'─'*80}")
    
    # Show all candidate models and their metrics
    print(f"\nModels evaluated: {list(cohort_results['candidate_results'].keys())}")
    print(f"Best model: {cohort_results['best_model_name']}")
    
    for model_name, model_results in cohort_results['candidate_results'].items():
        print(f"\n{'─'*60}")
        print(f"MODEL: {model_name}")
        if model_name == cohort_results['best_model_name']:
            print("⭐ SELECTED AS BEST MODEL")
        print(f"{'─'*60}")
        
        metrics = model_results['metrics']
        
        # Show train metrics
        print("\nTrain Metrics:")
        for k, v in metrics['train'].items():
            if k != 'lift_table':
                if isinstance(v, float):
                    print(f"  {k}: {v:.4f}")
                else:
                    print(f"  {k}: {v}")
        
        # Show OOT metrics
        print("\nOOT Metrics:")
        for k, v in metrics['oot'].items():
            if k != 'lift_table':
                if isinstance(v, float):
                    print(f"  {k}: {v:.4f}")
                else:
                    print(f"  {k}: {v}")
        
        # Show CV info if available
        if model_results['cv_results']['cv_enabled']:
            print("\nCV Results:")
            print(f"  Best CV score: {model_results['cv_results']['best_score']:.4f}")
            print(f"  Best params: {model_results['cv_results']['best_params']}")
    
    # ========================================================================
    # LIFT TABLE (Classification only)
    # ========================================================================
    if task_type == 'classification':
        print(f"\n{'='*80}")
        print(f"LIFT TABLE ANALYSIS")
        print(f"{'='*80}")
        
        MODEL_TO_EXAMINE = cohort_results['best_model_name']  # Or specify manually
        
        if MODEL_TO_EXAMINE in cohort_results['candidate_results']:
            lift_table = cohort_results['candidate_results'][MODEL_TO_EXAMINE]['metrics']['oot']['lift_table']
            
            print(f"\nModel: {MODEL_TO_EXAMINE}")
            print(f"Cohort: {COHORT_TO_EXAMINE}")
            print("\n" + lift_table.to_string(index=False))
            
            # Key insights
            print(f"\n{'─'*60}")
            print("Key Insights:")
            print(f"{'─'*60}")
            top_decile_lift = lift_table.iloc[0]['lift']
            top_decile_capture = lift_table.iloc[0]['cum_pct_captured']
            top_5pct = lift_table.iloc[:1]
            
            print(f"Top 5% lift: {top_decile_lift:.2f}x")
            print(f"Top 5% captures: {top_decile_capture:.1%} of all events")
            
            # Find where lift drops below 2x
            below_2x = lift_table[lift_table['cum_lift'] < 2.0]
            if len(below_2x) > 0:
                first_below = below_2x.iloc[0]
                pct = (first_below['quantile'] / 20) * 100
                print(f"Lift drops below 2x at: top {pct:.0f}%")
        else:
            print(f"⚠️  Model '{MODEL_TO_EXAMINE}' not found")
    
    # ========================================================================
    # SHAP ANALYSIS (Best model only)
    # ========================================================================
    print(f"\n{'='*80}")
    print(f"SHAP FEATURE IMPORTANCE")
    print(f"{'='*80}")
    
    if cohort_results['shap_values'] is not None:
        shap_dict = cohort_results['shap_values']
        
        print(f"\nModel: {cohort_results['best_model_name']}")
        print(f"Cohort: {COHORT_TO_EXAMINE}")
        
        # Generate SHAP direction summary
        shap_direction = summarize_shap_direction(shap_dict, top_n=20)
        
        if shap_direction is not None:
            print("\nTop Features by SHAP Importance:")
            print(shap_direction.to_string(index=False))
            
            # Plot SHAP summary
            print("\nGenerating SHAP summary plot...")
            output_dir = Path(OUTPUT_BASE_DIR) / LABEL / COHORT_TO_EXAMINE / 'best_model' / 'plots'
            plot_shap_summary(
                shap_dict,
                output_dir / 'shap_summary_detailed.png',
                top_n=20
            )
            print("✓ SHAP plot saved")
    else:
        print("⚠️  SHAP values not available (may be disabled or ensemble model)")
    
    # ========================================================================
    # THRESHOLD ANALYSIS (Classification only)
    # ========================================================================
    if task_type == 'classification' and cohort_results['threshold_results']:
        print(f"\n{'='*80}")
    # ========================================================================
    # COEFFICIENT TABLE (Logistic Regression only)
    # ========================================================================
    if cohort_results['coeff_table'] is not None:
        print(f"\n{'='*80}")
        print(f"LOGISTIC REGRESSION COEFFICIENTS")
        print(f"{'='*80}")
        
        print(f"\nModel: {cohort_results['best_model_name']}")
        print(f"Cohort: {COHORT_TO_EXAMINE}")
        
        coeff_table = cohort_results['coeff_table']
        
        print("\nTop Features by Absolute Coefficient:")
        print(coeff_table[['feature', 'coefficient', 'odds_ratio', 'interpretation']].to_string(index=False))
        
        # Summary stats
        print(f"\n{'─'*60}")
        print("Summary:")
        print(f"{'─'*60}")
        print(f"Total features: {len(coeff_table)}")
        
        positive_coef = (coeff_table['coefficient'] > 0).sum()
        negative_coef = (coeff_table['coefficient'] < 0).sum()
        print(f"Positive coefficients (increase odds): {positive_coef}")
        print(f"Negative coefficients (decrease odds): {negative_coef}")
        
        # Strongest effects
        max_positive = coeff_table.loc[coeff_table['coefficient'].idxmax()]
        max_negative = coeff_table.loc[coeff_table['coefficient'].idxmin()]
        
        print(f"\nStrongest positive effect: {max_positive['feature']}")
        print(f"  Odds ratio: {max_positive['odds_ratio']:.3f}")
        print(f"\nStrongest negative effect: {max_negative['feature']}")
        print(f"  Odds ratio: {max_negative['odds_ratio']:.3f}")
    
        print(f"THRESHOLD OPTIMIZATION")
        print(f"{'='*80}")
        
        print(f"\nModel: {cohort_results['best_model_name']}")
        print(f"Cohort: {COHORT_TO_EXAMINE}")
        
        print("\nOptimal Thresholds:")
        for obj_name, obj_result in cohort_results['threshold_results'].items():
            if obj_name != '_threshold_curve':
                print(f"\n{obj_name}:")
                print(f"  Threshold: {obj_result['threshold']:.3f}")
                for k, v in obj_result.items():
                    if k != 'threshold':
                        if isinstance(v, float):
                            print(f"  {k}: {v:.4f}")
                        else:
                            print(f"  {k}: {v}")
        
        # Show threshold curve stats
        if '_threshold_curve' in cohort_results['threshold_results']:
            df_curve = cohort_results['threshold_results']['_threshold_curve']
            
            print(f"\n{'─'*60}")
            print("Threshold Sensitivity:")
            print(f"{'─'*60}")
            
            # Show key thresholds
            thresholds_to_show = [0.1, 0.3, 0.5, 0.7, 0.9]
            print(f"{'Threshold':<12} {'Precision':<12} {'Recall':<12} {'F1':<12}")
            print("─" * 50)
            for thresh in thresholds_to_show:
                row = df_curve.iloc[(df_curve['threshold'] - thresh).abs().argmin()]
                print(f"{row['threshold']:<12.2f} {row['precision']:<12.3f} "
                      f"{row['recall']:<12.3f} {row['f1']:<12.3f}")
    
    # ========================================================================
    # PREDICTION DISTRIBUTION
    # ========================================================================
    print(f"\n{'='*80}")
    print(f"PREDICTION DISTRIBUTION")
    print(f"{'='*80}")
    
    print(f"\nModel: {cohort_results['best_model_name']}")
    print(f"Cohort: {COHORT_TO_EXAMINE}")
    
    oot_preds = cohort_results['best_predictions']['oot']
    
    if task_type == 'classification':
        pred_col = 'predicted'
        print("\nPredicted Probability Distribution:")
        print(f"  Min:    {oot_preds[pred_col].min():.4f}")
        print(f"  25th %: {oot_preds[pred_col].quantile(0.25):.4f}")
        print(f"  Median: {oot_preds[pred_col].median():.4f}")
        print(f"  75th %: {oot_preds[pred_col].quantile(0.75):.4f}")
        print(f"  Max:    {oot_preds[pred_col].max():.4f}")
        
        # Show actual event rate by score bucket
        try:
            oot_preds['score_bucket'] = pd.qcut(
                oot_preds[pred_col], 
                q=10, 
                labels=False,  # Use numeric labels instead
                duplicates='drop'
            )
            
            # Convert to decile labels
            unique_buckets = sorted(oot_preds['score_bucket'].unique())
            bucket_map = {old: f'D{i+1}' for i, old in enumerate(unique_buckets)}
            oot_preds['score_bucket'] = oot_preds['score_bucket'].map(bucket_map)
            
            bucket_stats = oot_preds.groupby('score_bucket').agg({
                'actual': ['count', 'mean'],
                'predicted': 'mean'
            }).round(4)
            
            print("\nActual Event Rate by Score Bucket:")
            print(f"(Created {len(unique_buckets)} buckets)")
            print(bucket_stats.to_string())
            
        except Exception as e:
            print(f"\n⚠️  Could not create score buckets: {e}")
            print("Scores may have too many duplicate values")
        
    else:
        pred_col = 'predicted'
        print("\nPredicted Value Distribution:")
        print(f"  Min:    {oot_preds[pred_col].min():.2f}")
        print(f"  25th %: {oot_preds[pred_col].quantile(0.25):.2f}")
        print(f"  Median: {oot_preds[pred_col].median():.2f}")
        print(f"  75th %: {oot_preds[pred_col].quantile(0.75):.2f}")
        print(f"  Max:    {oot_preds[pred_col].max():.2f}")
        
        print("\nActual Value Distribution:")
        print(f"  Min:    {oot_preds['actual'].min():.2f}")
        print(f"  25th %: {oot_preds['actual'].quantile(0.25):.2f}")
        print(f"  Median: {oot_preds['actual'].median():.2f}")
        print(f"  75th %: {oot_preds['actual'].quantile(0.75):.2f}")
        print(f"  Max:    {oot_preds['actual'].max():.2f}")

print(f"\n{'='*80}")
print(f"END OF DETAILED EXAMINATION")
print(f"{'='*80}")


RESULTS SUMMARY

Label: label_will_give_during_giving_tuesday
Task: classification

    cohort best_model  n_features  train_auc  oot_auc  train_pr_auc  oot_pr_auc  oot_brier  lift_at_1pct  lift_at_5pct  lift_at_10pct
all_donors     logreg          56   0.864339 0.858392      0.259752    0.215496   0.021928     14.886795      8.671313       5.970988

TOP FEATURES BY COHORT

all_donors:
                             feature  importance
                  email_open_rate_3m          94
                days_since_last_gift          83
teacher_still_available_during_range          42
  email_open_rate_velocity_3m_vs_12m          42
           pct_amount_green_lifetime          41
                          is_teacher          19
     latest_project_got_fully_funded          17
         latest_gift_amount_nongreen          17
            pct_amount_big_event_12m          15
             pct_gifts_gift_card_12m          14

THRESHOLD RECOMMENDATIONS

all_donors:
  f1_optimal: 0.198
  precision

---
## Next Steps

1. **Review metrics**: Check the summary table above
2. **Inspect features**: Look at top SHAP features
3. **Examine plots**: Open the plots directory for visualizations
4. **Use model**: Load the saved model.pkl to score new donors
5. **Iterate**: Adjust configs and re-run with different labels/cohorts

### To score new donors:
```python
# Load best model
with open('./models/<label>/<cohort>/best_model/model.pkl', 'rb') as f:
    model = pickle.load(f)

# Load config to get feature names
with open('./models/<label>/<cohort>/best_model/config.json', 'r') as f:
    config = json.load(f)

# Score new data (must have same features)
new_data = pd.read_csv('new_donors.csv')
new_data_features = new_data[config['feature_names']]
predictions = model.predict_proba(new_data_features)[:, 1]
```